In [1]:
import os
import sys

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from scipy import ndimage

from six.moves import cPickle as pickle

import tensorflow as tf

In [15]:
root='/home/cc/Data/stanford_car/'
pickle_file = 'train_dataset_1000_1500.pickle'
train_dataset = pickle.load( open( pickle_file, "rb" ) )
print(type(train_dataset))

<type 'dict'>


In [16]:
data = train_dataset['data']
labels = train_dataset['labels']

In [4]:
image_size = 227
image_channels = 3
num_classes = 196

sess = tf.InteractiveSession()

x = tf.placeholder(tf.float32,[None, image_size, image_size, image_channels])
y = tf.placeholder(tf.float32,[None, num_classes])

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W, strides, padding):
    return tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding=padding)

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='VALID')

def Normalization(x):
    Norm=tf.nn.local_response_normalization(x, depth_radius=5, bias=1.0, alpha=0.0001, beta=0.75, name=None)
    return Norm


In [5]:

layer1_conv = {'in_size': (227,227), 'dept': 3, 'out_size': (55,55), 'fsize': (11, 11), 'nof':96, 'pad': 'VALID', 'strid': 4}
layer1_maxpool = {'in_size': (55,55), 'dept': 96, 'out_size': (27,27), 'fsize': (3, 3), 'strid': 2, 'pad': 'VALID'}

layer2_conv = {'in_size': (27,27), 'dept': 96, 'out_size': (23,23), 'fsize': (5, 5), 'nof':256, 'pad': 'VALID', 'strid': 1}
layer2_maxpool = {'in_size': (23,23), 'dept': 256, 'out_size': (11,11), 'fsize': (3, 3), 'strid': 2, 'pad': 'VALID'}

layer3_conv = {'in_size': (11,11), 'dept': 256, 'out_size': (9,9), 'fsize': (3, 3), 'nof':384, 'pad': 'VALID', 'strid': 1}

layer4_conv = {'in_size': (9,9), 'dept': 384, 'out_size': (7,7), 'fsize': (3, 3), 'nof':384, 'pad': 'VALID', 'strid': 1}

layer5_conv = {'in_size': (7,7), 'dept': 384, 'out_size': (5,5), 'fsize': (3, 3), 'nof':256, 'pad': 'VALID', 'strid': 1}
layer5_maxpool = {'fsize': (3, 3), 'strid': 2, 'pad': 'VALID'}

layer6_fc = {'dept':4*4*256, 'out_size': 4096}

layer7_fc = {'dept':4096, 'out_size': 4096}

layer8_fc = {'dept':4096, 'out_size': 196}


In [6]:

#Layer 1#
W_conv1 = weight_variable([layer1_conv['fsize'][0], layer1_conv['fsize'][1], layer1_conv['dept'], layer1_conv['nof']])
b_conv1 = bias_variable([layer1_conv['nof']])
c_conv1 = tf.nn.conv2d(x, W_conv1, strides=[1, layer1_conv['strid'], layer1_conv['strid'], 1], padding=layer1_conv['pad'])
h_conv1 = tf.nn.relu(c_conv1 + b_conv1)
Norm1=Normalization(h_conv1)
h_pool1 = max_pool_2x2(Norm1)                                        

#Layer 2#
W_conv2 = weight_variable([layer2_conv['fsize'][0], layer2_conv['fsize'][1], layer2_conv['dept'], layer2_conv['nof']])
b_conv2 = bias_variable([layer2_conv['nof']])
c_conv2 = tf.nn.conv2d(h_pool1, W_conv2, strides=[1, layer2_conv['strid'], layer2_conv['strid'], 1], padding=layer2_conv['pad'])
h_conv2 = tf.nn.relu(c_conv2 + b_conv2)
Norm2=Normalization(h_conv2)
h_pool2 = max_pool_2x2(Norm2)                                        

#Layer 3#
W_conv3 = weight_variable([layer3_conv['fsize'][0], layer3_conv['fsize'][1], layer3_conv['dept'], layer3_conv['nof']])
b_conv3 = bias_variable([layer3_conv['nof']])
c_conv3 = tf.nn.conv2d(h_pool2, W_conv3, strides=[1, layer3_conv['strid'], layer3_conv['strid'], 1], padding=layer3_conv['pad'])
h_conv3 = tf.nn.relu(c_conv3 + b_conv3)

#Layer 4#
W_conv4 = weight_variable([layer4_conv['fsize'][0], layer4_conv['fsize'][1], layer4_conv['dept'], layer4_conv['nof']])
b_conv4 = bias_variable([layer4_conv['nof']])
c_conv4 = tf.nn.conv2d(h_conv3, W_conv4, strides=[1, layer4_conv['strid'], layer4_conv['strid'], 1], padding=layer4_conv['pad'])
h_conv4 = tf.nn.relu(c_conv4 + b_conv4)

#Layer 5#
W_conv5 = weight_variable([layer5_conv['fsize'][0], layer5_conv['fsize'][1], layer5_conv['dept'], layer5_conv['nof']])
b_conv5 = bias_variable([layer5_conv['nof']])
c_conv5 = tf.nn.conv2d(h_conv4, W_conv5, strides=[1, layer5_conv['strid'], layer5_conv['strid'], 1], padding=layer5_conv['pad'])
h_conv5 = tf.nn.relu(c_conv5 + b_conv5)
h_pool5 = max_pool_2x2(h_conv5)                                      

#Layer 6 - FC1#
in_shape = h_pool5.get_shape()
in_size = in_shape.as_list()[1]*in_shape.as_list()[2]*in_shape.as_list()[3]

W_fc1 = weight_variable([in_size, layer6_fc['out_size']])
b_fc1 = bias_variable([layer6_fc['out_size']])

h_pool5_flat = tf.reshape(h_pool5, [-1, in_size])
h_fc1 = tf.nn.relu(tf.matmul(h_pool5_flat, W_fc1) + b_fc1)
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

#Layer 7 - FC2#
W_fc2 = weight_variable([layer6_fc['out_size'], layer7_fc['out_size']])
b_fc2 = bias_variable([layer7_fc['out_size']])
h_fc2 = tf.nn.relu(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)
h_fc2_drop = tf.nn.dropout(h_fc2, keep_prob)

#Layer 8 - FC3#
W_fc3 = weight_variable([layer7_fc['out_size'], layer8_fc['out_size']])
b_fc3 = bias_variable([layer8_fc['out_size']])
y_predict = tf.nn.softmax(tf.matmul(h_fc2_drop, W_fc3) + b_fc3)


In [7]:
#Optimization Operation
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y_predict, y))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y_predict,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


In [ ]:
batch_size = 50
nofsteps = 5000

sess.run(tf.initialize_all_variables())
graph_accuracy = np.zeros(nofsteps)

for step in range(nofsteps):
    offset = (step * batch_size) % (labels.shape[0] - batch_size)
    batch_data = data[offset:(offset + batch_size), :, :]
    batch_labels = labels[offset:(offset + batch_size)]
    graph_accuracy[step] = accuracy.eval(feed_dict={x: batch_data, y: batch_labels, keep_prob: 1.0})
    if step%100 == 0:
        train_accuracy = accuracy.eval(feed_dict={x: batch_data, y: batch_labels, keep_prob: 1.0})
        print("step %d, training accuracy %g"%(step, train_accuracy))
    train_step.run(feed_dict={x: batch_data, y: batch_labels, keep_prob: 0.5})

step 0, training accuracy 0.02
